In [ ]:
# Mounting Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Installing Dependencies

!pip install xformers
!pip install transformers[torch]
!pip install datasets
!pip install  torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
# Loading Required Libraries

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
import time
import copy

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import math
import random
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import gc
import torchtext
from tqdm import tqdm

# Misc.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Hyperparameter Values for Training

batch_size = 16
MAX_LEN = 160
RANDOM_SEED = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 20


In [ ]:
# Checking device type

device

device(type='cuda')

# Dataset

In [ ]:
# Loading the dataset from drive into a Pandas dataframe

df = pd.read_csv("/content/drive/MyDrive/NLP4RE-data/Expanded_PROMISE.csv")

In [ ]:
# Checking first 10 rows of the dataframe

df.head(10)

,Id,RequirementText,Class,Binary_Label
0,1,The product shall be available during normal b...,A,1
1,2,The product shall be available for use 24 hour...,A,1
2,2,Out of 1000 accesses to the system the system ...,A,1
3,3,The system shall be available for use between ...,A,1
4,3,The system shall achieve 95% up time.,A,1
5,5,The product shall adhere to the corporate onli...,A,1
6,5,The product shall achieve a 98% uptime. The pr...,A,1
7,6,Aside from server failure the software produc...,A,1
8,8,The website shall be available for use 24 hour...,A,1
9,8,The website shall achieve 99.5% up time.,A,1


In [ ]:
# Dataframe metadata information

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Id               969 non-null    int64 
 1   RequirementText  969 non-null    object
 2   Class            969 non-null    object
 3   Binary_Label     969 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 30.4+ KB


In [ ]:
# Distribution of the dataframe

df['Binary_Label'].value_counts()

1    525
0    444
Name: Binary_Label, dtype: int64

In [ ]:
# Creating a train and validation set from data

df_train, df_val= train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Distribution of train and validation set

df_train['Binary_Label'].value_counts() , df_val['Binary_Label'].value_counts()

(1    414
 0    361
 Name: Binary_Label, dtype: int64,
 1    111
 0     83
 Name: Binary_Label, dtype: int64)

In [ ]:
df2 = df.copy()

def map_label(label):
    if label == 0:
        return 'function'
    elif label == 1:
        return 'non-function'
    else:
        return 'unknown'

# Apply the mapping function to create a new column
df2['category'] = df2["Binary_Label"].apply(map_label)

In [ ]:
df_train2, df_val2= train_test_split(df2, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
trg_langs = df_train2["category"].unique().tolist()
trg_langs

['function', 'non-function']

In [ ]:
class LangDataset(Dataset):
    def __init__(self, ds, trg_langs, train_vocab=None):
        self.corpus = ds

        if not train_vocab:
            self.src_vocab, self.trg_vocab = self._build_vocab()
        else:
            self.src_vocab, self.trg_vocab = train_vocab

    def __len__(self):
        return len(self.corpus)

    def __getitem__(self, item):
        text = self.corpus.iloc[item].RequirementText
        lang = self.corpus.iloc[item].category

        return {
            'src': self.src_vocab.lookup_indices(text.lower().split()),
            'trg': self.trg_vocab.lookup_indices([lang])
        }

    def _build_vocab(self):
        # Here one could remove stopwords and use word lemmatisation.
        # Both techniques will reduce the vocab size and hence model size
        # and could also enhance the model's performance
        src_tokens = self.corpus.RequirementText.str.cat().lower().split()

        src_vocab = build_vocab_from_iterator([src_tokens], specials=["<unk>", "<pad>"])
        src_vocab.set_default_index(src_vocab['<unk>'])

        trg_vocab = build_vocab_from_iterator([trg_langs])

        return src_vocab, trg_vocab

In [ ]:
def collate_fn(batch, pad_value, device):
    trgs = []
    srcs = []
    for row in batch:
        srcs.append(torch.tensor(row["src"], dtype=torch.long).to(device))
        trgs.append(torch.tensor(row["trg"]).to(device))

    padded_srcs = pad_sequence(srcs, padding_value=pad_value)
    return {"src": padded_srcs, "trg": torch.tensor([trgs]).to(device)}

train_langds = LangDataset(df_train2, trg_langs)
test_langds = LangDataset(df_val2, trg_langs, (train_langds.src_vocab, train_langds.trg_vocab))

SRC_PAD_IDX = train_langds.src_vocab["<pad>"]

train_dt = DataLoader(train_langds, batch_size= batch_size, shuffle=
                   True, collate_fn=lambda batch_size: collate_fn(batch_size, SRC_PAD_IDX, device))

test_dt = DataLoader(test_langds, batch_size=batch_size, shuffle=
                   True, collate_fn=lambda batch_size: collate_fn(batch_size, SRC_PAD_IDX, device))


hyp_params = {
    "batch_size": batch_size,
    "lr": 0.00001,
    "num_epochs": EPOCHS,
    "d_model": 512, # Input embedding dimension
    "n_head": 8, # No. of multi-head attention block (aka paralle self-attention layers)
    "n_layers": 3,
    "feedforward_dim": 128,
    "dropout": 0.1
}


hyp_params["src_vocab_size"] = len(train_langds.src_vocab)
hyp_params["trg_vocab_size"] = len(trg_langs)

In [ ]:
next(iter(train_dt))

{'src': tensor([[ 163,  103,   81,    2,    2,    2,   53,    2,   83,   23,    2,    2,
             2,    7,   12,    2],
         [   2,   26,   98,   10,   46,    7,  112,   10,    2,    3,   10,    7,
             7,   15, 1315,    7],
         [ 392,   45,    9,    3,    3,   15,   12,    3,  188,  252,    3,    3,
             3,   21,    9,    3],
         [   4,  109,   80,  396,  222,  105,    3,    6,  155, 1022,  171,  163,
            29,   78,  970,   29],
         [ 253,   87,   62,    2,  126,  326,   39,  165,   13,   76,    8,   12,
           211,    4,   44,    2],
         [ 637,   63,   75, 2052,   12,  357,    2,  758,  659,    2,   56,    4,
            10, 2081,  545, 1662],
         [  84, 1146,    3,  123, 1326,   83,  228,  879,    8,  972,   31,  920,
          1391,   11,  937,  178],
         [ 787,   94,    6,   25,    5,  113,  235,   10,  311, 1184,  317,   99,
             4,    8,    2,  367],
         [  24,   11,   16,    4,  126,  193,   53,   14,

# Transformers

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, maxlen = 5000):
        super(PositionalEncoding, self).__init__()

        # A tensor consists of all the possible positions (index) e.g 0, 1, 2, ... max length of input
        # Shape (pos) --> [max len, 1]
        pos = torch.arange(0, maxlen).unsqueeze(1)
        pos_encoding = torch.zeros((maxlen, d_model))

        sin_den = 10000 ** (torch.arange(0, d_model, 2)/d_model) # sin for even item of position's dimension
        cos_den = 10000 ** (torch.arange(1, d_model, 2)/d_model) # cos for odd

        pos_encoding[:, 0::2] = torch.sin(pos / sin_den)
        pos_encoding[:, 1::2] = torch.cos(pos / cos_den)

        # Shape (pos_embedding) --> [max len, d_model]
        # Adding one more dimension in-between
        pos_encoding = pos_encoding.unsqueeze(-2)
        # Shape (pos_embedding) --> [max len, 1, d_model]

        self.dropout = nn.Dropout(dropout)

        # We want pos_encoding be saved and restored in the `state_dict`, but not trained by the optimizer
        # hence registering it!
        # Source & credits: https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter-of-nn-module/32723/2
        self.register_buffer('pos_encoding', pos_encoding)

    def forward(self, token_embedding):
        # shape (token_embedding) --> [sentence len, batch size, d_model]

        # Concatenating embeddings with positional encodings
        # Note: As we made positional encoding with the size max length of sentence in our dataset
        #       hence here we are picking till the sentence length in a batch
        #       Another thing to notice is in the Transformer's paper they used FIXED positional encoding,
        #       there are methods where we can also learn them
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])


class InputEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(InputEmbedding, self).__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, tokens):
        # shape (tokens) --> [sentence len, batch size]
        # shape (inp_emb) --> [sentence len, batch size, d_model]
        # Multiplying with square root of d_model as they mentioned in the Transformer's paper
        inp_emb = self.embedding(tokens.long()) * math.sqrt(self.d_model)
        return inp_emb


class TransformerClassifier(nn.Module):
    def __init__(self,
                  src_vocab_size,
                 trg_vocab_size ,
                 d_model,
                 dropout,
                 n_head,
                 dim_feedforward,
                 n_layers,
                ):
        super().__init__()

        self.src_inp_emb = InputEmbedding(src_vocab_size, d_model)
        self.trg_inp_emb = InputEmbedding(trg_vocab_size, d_model)

        self.positional_encoding = PositionalEncoding(d_model, dropout=dropout)

        # Only using Encoder of Transformer model
        encoder_layers = nn.TransformerEncoderLayer(d_model, n_head, dim_feedforward, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)

        self.d_model = d_model
        self.decoder = nn.Linear(d_model, trg_vocab_size)

    def forward(self, x):
        x_emb = self.positional_encoding(self.src_inp_emb(x))
        # Shape (output) -> (Sequence length, batch size, d_model)
        output = self.transformer_encoder(x_emb)
        # We want our output to be in the shape of (batch size, d_model) so that
        # we can use it with CrossEntropyLoss hence averaging using first (Sequence length) dimension
        # Shape (mean) -> (batch size, d_model)
        # Shape (decoder) -> (batch size, d_model)
        return self.decoder(output.mean(0))

In [ ]:
def train_model(model, train_dataloader, criterion, optimizer , n_examples ):
    model.train()
    epoch_loss = 0
    correct_predictions = 0
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        # Clear the accumulating gradients
        optimizer.zero_grad()

        src = batch["src"]  # shape --> [seq len, batch size]
        trg = batch["trg"]  # shape --> [1, batch size]

        # shape (out) --> [batch size, trg size]
        out = model(src)
        _, preds = torch.max(out, dim=1)
        correct_predictions += torch.sum(preds == trg)
        loss = criterion(out, trg.squeeze(0))

        loss.backward()

        optimizer.step()
        epoch_loss += loss.detach().cpu()

    return correct_predictions.double() / n_examples, epoch_loss/len(train_dataloader)


def evaluate_model(model, valid_dataloader, criterion , n_examples):
    model.eval()
    epoch_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_dataloader):
            src = batch["src"]  # shape --> [seq len, batch size]
            trg = batch["trg"]  # shape --> [1, batch size]

            # shape (out) --> [batch size, trg size]
            out = model(src)
            _, preds = torch.max(out, dim=1)
            correct_predictions += torch.sum(preds == trg)
            loss = criterion(out, trg.squeeze(0))

            epoch_loss += loss.detach().cpu()



    return correct_predictions.double() / n_examples, epoch_loss/len(valid_dataloader)

In [ ]:
model2 = TransformerClassifier(  hyp_params["src_vocab_size"],
                                hyp_params["trg_vocab_size"] ,
                                hyp_params["d_model"],
                                hyp_params["dropout"],
                                hyp_params["n_head"],
                                hyp_params["feedforward_dim"],
                                hyp_params["n_layers"]
                                ).to(device)


criterion = nn.CrossEntropyLoss().to(device)

optimizer = optim.Adam(model2.parameters(), lr=hyp_params["lr"])

print(model2)

TransformerClassifier(
  (src_inp_emb): InputEmbedding(
    (embedding): Embedding(2602, 512)
  )
  (trg_inp_emb): InputEmbedding(
    (embedding): Embedding(2, 512)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear

In [ ]:
hyp_params

{'batch_size': 16,
 'lr': 1e-05,
 'num_epochs': 20,
 'd_model': 512,
 'n_head': 8,
 'n_layers': 3,
 'feedforward_dim': 128,
 'dropout': 0.1,
 'src_vocab_size': 2602,
 'trg_vocab_size': 2}

In [ ]:
def train(model,EPOCHS = EPOCHS):


  for epoch in range(EPOCHS):
    start = time.time()
    gc.collect()
    torch.cuda.empty_cache()

    train_acc,train_loss = train_model(model, train_dt, criterion, optimizer,len(df_train2))
    val_acc,val_loss = evaluate_model(model, test_dt, criterion,len(df_val2))

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    print(f' train_loss {train_loss:.5f}  train_acc {train_acc:.5f} <-> Val_loss {val_loss:.5f} val_accuracy {val_acc:.5f}')

In [ ]:
train(model2, 20)


# {'batch_size': 16,
#  'lr': 0.0005,
#  'num_epochs': 40,
#  'd_model': 768,
#  'n_head': 12,
#  'n_layers': 12,
#  'feedforward_dim': 128,
#  'dropout': 0.1,
#  'src_vocab_size': 2602,
#  'trg_vocab_size': 2}

100%|██████████| 49/49 [00:00<00:00, 61.29it/s]


Epoch 1/20
----------
 train_loss 0.69208  train_acc 0.54710 <-> Val_loss 0.68288 val_accuracy 0.52577


100%|██████████| 49/49 [00:00<00:00, 66.18it/s]


Epoch 2/20
----------
 train_loss 0.65471  train_acc 0.62710 <-> Val_loss 0.63298 val_accuracy 0.61856


100%|██████████| 49/49 [00:00<00:00, 70.29it/s]


...

Epoch 18/20
----------
 train_loss 0.18337  train_acc 0.93677 <-> Val_loss 0.47698 val_accuracy 0.82990


100%|██████████| 49/49 [00:00<00:00, 69.86it/s]


Epoch 19/20
----------
 train_loss 0.18520  train_acc 0.92903 <-> Val_loss 0.44381 val_accuracy 0.83505


100%|██████████| 49/49 [00:00<00:00, 58.60it/s]


Epoch 20/20
----------
 train_loss 0.15384  train_acc 0.95097 <-> Val_loss 0.54295 val_accuracy 0.83505


# Unlabeled Data

In [ ]:
import zipfile
import os

# Path to the zip file
zip_file_path = "/content/drive/MyDrive/NLP4RE-data/pure-requirements-xml.zip"

# Directory to extract the files
extract_dir = "/content/drive/MyDrive/NLP4RE-data"

# # Open the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     # Extract all the files into the directory specified by extract_dir
#     zip_ref.extractall(extract_dir)

extracted_path = os.path.join(extract_dir, "pure-requirements-xml")
# Iterate through each file in the directory
for f in os.listdir(extracted_path):
  # join path
  extracted_file_path = os.path.join(extracted_path, f)
  # Open and read the contents of the extracted file
  with open(extracted_file_path, 'r') as f1:
      contents = f1.read()
      print(contents)
      break

/content/drive/MyDrive/NLP4RE-data/pure-requirements-xml/2007-ertms.xml
<?xml version="1.0" encoding="UTF-8"?>
<!-- New document created with EditiX at Tue Jun 07 14:21:44 CEST 2011 -->
<req_document xsi:schemaLocation="req_document.xsd req_document.xsd" 
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
xmlns="req_document.xsd">

<title>ERTMS/ETCS Functional Requirements Specification FRS</title>
<version>5.00</version>
<issue_date>2007-06-21</issue_date>
<file_number>ERA/ERTMS/003204</file_number>
<change_log>
	<change_log_item>
	<version></version>
	<change></change>
	<change_date>2007-06-21</change_date>
	</change_log_item>
</change_log>
<p id="1">
<title>Introduction</title>
<p id="1.1">
<text_body>
This document defines the functional requirements for ERTMS/ETCS (EUROPEAN RAIL TRAFFIC MANAGEMENT SYSTEM / EUROPEAN TRAIN CONTROL SYSTEM). The document primarily defines the operational requirements and therefore contains only a few technical terms. For consistency reasons, all f